In [ ]:
%pylab inline
import pandas as pd
import re

m_2009 = pd.read_csv('movies_2009_2018_df.csv')
m_1990 = pd.read_csv('movies_1990_2008.csv')
raw_dfs = [m_2009, m_1990]
clean_dfs = []
for df in raw_dfs:
    df = df.dropna(subset=['imdbID', 'BoxOffice'])
    df['imdbID'] = df['imdbID'].apply(lambda x: int(x[2:]))
    df = df.set_index('imdbID')
    df = df[df.index.duplicated()==False]
    clean_dfs.append(df)
    
m_all = pd.concat(clean_dfs,axis=0)
m_all = m_all[m_all.index.duplicated()==False]

'''Processing'''

metadata = pd.read_csv('movies_metadata.csv', low_memory=False).dropna(subset=['imdb_id'])
metadata = metadata[metadata['imdb_id'].str.contains('tt')]
metadata['imdb_id'] = metadata['imdb_id'].apply(lambda x: int(x[2:]))

metadata = metadata.set_index(metadata['imdb_id'], drop=True)

metadata = metadata[metadata['budget'].str.isnumeric()]

to_float = ['revenue','budget']
for c in to_float:
    try:
        metadata = metadata[metadata[c].str.isnumeric()]
    except:
        #is already numeric
        pass
    metadata[c] = metadata[c].apply(lambda x: float(x))
    metadata = metadata[metadata[c]>0]

metadata['profit'] = metadata['revenue']-metadata['budget']

metadata = metadata[metadata.index.duplicated()==False]


intersection = m_all['BoxOffice'].index.intersection(metadata['profit'].index)
box_office_profit = pd.concat([m_all['BoxOffice'].loc[intersection], metadata[['profit','id']].loc[intersection]], axis=1)
box_office_profit['BoxOffice'] = box_office_profit['BoxOffice'].apply(lambda x: int(re.sub('[^0-9]','',x)))
b_idx = box_office_profit['id'].apply(int)


box_office_profit = box_office_profit.set_index(b_idx)

ratings = pd.read_csv('ratings.csv')

r = ratings.groupby('movieId').mean()['rating']

b_idx = box_office_profit.index
r_idx = r.index
intersection = b_idx.intersection(r_idx)

b_p_r_df = pd.concat([box_office_profit.loc[intersection], r.loc[intersection]], axis=1)[['BoxOffice','profit','rating']]

b_p_r_df.to_csv('box_office_profit_rating.csv')